<a href="https://colab.research.google.com/github/vermacularis/AI-Powered-Resume-Screening-Tool/blob/main/AI_Powered_Resume_Screening_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install spacy scikit-learn pandas PyPDF2

!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import os
import pandas as pd
import spacy
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [5]:
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


In [6]:
nlp = spacy.load("en_core_web_sm")


In [7]:
def extract_resume_data(text):
    doc = nlp(text)
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


In [8]:
def rank_resumes(job_description, resume_texts):
    all_texts = [job_description] + resume_texts
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(all_texts)
    scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    return scores


In [11]:
from google.colab import files
uploaded = files.upload()
resume_files = list(uploaded.keys())


Saving Resume ARPIT VERMA.pdf to Resume ARPIT VERMA.pdf


In [12]:
job_description = "We are hiring a data scientist with skills in Python, machine learning, and data analysis."

resume_texts = []
for file in resume_files:
    raw_text = extract_text_from_pdf(file)
    clean_text = extract_resume_data(raw_text)
    resume_texts.append(clean_text)

scores = rank_resumes(job_description, resume_texts)

results = pd.DataFrame({
    "Resume": resume_files,
    "Score": scores
}).sort_values(by="Score", ascending=False)

results.reset_index(drop=True, inplace=True)
results


,Resume,Score
0,Resume ARPIT VERMA.pdf,0.074777


In [13]:
results.to_csv("ranked_resumes.csv", index=False)
files.download("ranked_resumes.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>